In [58]:
import numpy as np
import math

In [59]:
class TicTacToe:
    def __init__(self):
        self.row_count = 3
        self.column_count = 3
        self.action_size = self.row_count * self.column_count

    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    def get_next_state(self, state, action, player):
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = player
        return state
    
    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
    
    def check_win(self, state, action):
        if action == None:
            return False

        row = action // self.column_count
        column = action % self.column_count
        player = state[row, column]

        return (
            np.sum(state[row, :]) == player * self.column_count 
            or np.sum(state[:, column]) == player * self.row_count
            or np.sum(np.diag(state)) == player * self.row_count 
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count
        )
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        
        return 0, False
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player

In [60]:
class Node:
    def __init__(self, game, args, state, parent = None, action_taken = None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken

        self.children = []
        self.expandable = game.get_valid_moves(state)

        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        return np.sum(self.expandable) == 0 and len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb

        return best_child
    
    def get_ucb(self, child):
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)
    
    def expand(self):
        action = np.random.choice(np.where(self.expandable == 1)[0])
        self.expandable[action] = 0

        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action, 1)
        child_state = self.game.change_perspective(child_state, player=-1)

        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)

        return child
    
    def simulate(self):
        value, is_terminated = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)
        
        if is_terminated:
            return value
        
        rollout_state = self.state.copy()
        rollout_player = 1

        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])  # Changed from self.expandable to valid_moves
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminated = self.game.get_value_and_terminated(rollout_state, action)

            if is_terminated:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                
                return value
            
            rollout_player = self.game.get_opponent(rollout_player)

    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1

        value = self.game.get_opponent_value(value)
        
        if self.parent is not None:
            self.parent.backpropagate(value)


class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state):
        root = Node(self.game, self.args, state)

        for search in range(self.args['num_searches']):
            node = root

            while node.is_fully_expanded():
                node = node.select()

            value, is_terminated = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)

            if not is_terminated:
                node = node.expand()
                value = node.simulate()
            
            node.backpropagate(value)

        action_probs = np.zeros(self.game.action_size)
        
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count

        action_probs /= np.sum(action_probs)

        return action_probs

In [61]:
tictactoe = TicTacToe()

player = 1

args = {
    'C' : math.sqrt(2),
    'num_searches' : 1000
}

mcts = MCTS(tictactoe, args)

state = tictactoe.get_initial_state()

while True:
    print(state)

    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        print("valid_moves", [i for i in range (tictactoe.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}:"))


        if valid_moves[action] == 0:
            print("action is invalid")
            continue
        
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)
        
    state = tictactoe.get_next_state(state, action, player)

    value, is_terminated = tictactoe.get_value_and_terminated(state, action)

    if is_terminated:
        print(state)
        
        if value == 1:
            print(player, "won the game")
        else: 
            print("the game has ended with a draw")
        
        break
    
    player = tictactoe.get_opponent(player)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid_moves [0, 1, 2, 3, 4, 5, 6, 7, 8]
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
valid_moves [1, 2, 3, 5, 6, 7, 8]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
[[ 1.  0.  0.]
 [ 0. -1. -1.]
 [ 0.  0.  1.]]
valid_moves [1, 2, 3, 6, 7]
[[ 1.  0.  0.]
 [ 1. -1. -1.]
 [ 0.  0.  1.]]
[[ 1.  0.  0.]
 [ 1. -1. -1.]
 [-1.  0.  1.]]
valid_moves [1, 2, 7]
action is invalid
[[ 1.  0.  0.]
 [ 1. -1. -1.]
 [-1.  0.  1.]]
valid_moves [1, 2, 7]
[[ 1.  0.  1.]
 [ 1. -1. -1.]
 [-1.  0.  1.]]
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [-1.  0.  1.]]
valid_moves [7]
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [-1.  1.  1.]]
the game has ended with a draw
